In [ ]:
# PySpark
from pyspark.sql import SparkSession
import os

os.environ["PYSPARK_PYTHON"]="/home/pc/g5_env/bin/python39"

In [ ]:
# Schema declaration
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
#map reduce i think
from functools import reduce
#pyspark sql functions
from pyspark.sql import functions as f
# storage level
from pyspark import StorageLevel

In [ ]:
spark = SparkSession.builder.master("local[18]")\
            .appName("Read CSV")\
            .config('spark.executor.memory', '40g')\
            .config('spark.driver.maxResultSize', '20g')\
            .config('spark.driver.memory', '40g')\
            .getOrCreate()
sparkContext = spark.sparkContext
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 19:22:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark.conf.get('spark.sql.files.maxPartitionBytes')

'134217728b'

In [ ]:
spark.conf.set('spark.sql.files.maxPartitionBytes', '256000000b')
spark.conf.get('spark.sql.files.maxPartitionBytes')

'256000000b'

In [ ]:
df = spark.read.option("wholetext",False)\
    .text("/home/pc/ozj/wiki-data-en-csv-test-remove-html.txt")

In [ ]:
df.count()

453392

In [ ]:
# clear empty rows
df = df.dropna(how='any').where(df.value != ' ')

In [ ]:
# sampling 10% of dataset estinmate 10% * 16.6GB = 1.66GB
df2 = df.sample(0.1,seed=123)

In [ ]:
df2.show(5,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df2.count()

45564

In [ ]:
df2 = df2.dropna(how='any').where(df.value != ' ')

In [ ]:
df2.rdd.getNumPartitions()

70

In [ ]:
df2 = df2.repartition(90)

In [ ]:
df2.rdd.getNumPartitions()

90

### <font color='red'>Sentence Tokenize</font>

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
def sentence_tokenize(text, index):
    return index,sent_tokenize(text)

def f(x): return x

In [ ]:
from pyspark.sql.functions import col, explode, regexp_replace, length

new_df = df2.select("value") \
    .where(length(col("value")) >= 900)
new_df.count()

39441

In [ ]:
df2.count()

45564

In [ ]:
# Adding indexes to all sentence
# Swapping the column
# Splitting the sentence
RDD1 = df2.rdd.map(lambda x: x[0]).zipWithIndex()\
.map(lambda x: sentence_tokenize(x[0], x[1]))\
.flatMapValues(f)\
.persist(StorageLevel.MEMORY_ONLY)
# .toDF().show(100, truncate = False)

Remove html code and header (== like this ==)

## <font color='red'>Text Cleaning</font>


#### Remove HTML tag

In [ ]:
def remove_html_tags(text):
    #Remove html tags from a string
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, ' ', text)

#### Remove contractions

In [ ]:
import contractions

def contractions_word(text):
    # using contractions.fix to expand the shortened words
    try:
        text = contractions.fix(text)
    except:
        return text
    return text
# RDD = dataDF.rdd.map(lambda x: x[0]).zipWithIndex()\
# .map(lambda x: sentence_tokenize(x[0], x[1]))\
# .flatMapValues(f)\
# .map(lambda x : remove_html_tags(x[1]))\
# .map(lambda x : contractions_word(x))\
# .map(lambda x: remove_emoticons(x))\
# .map(lambda x : sent_tokenize(x))\
# .persist(StorageLevel.MEMORY_ONLY)

#### Remove Emoticons

In [ ]:
from emot.emo_unicode import EMOTICONS_EMO

def remove_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, '')
    return text

#RDD2 = RDD.map(lambda x: remove_emoticons(x)).persist(StorageLevel.MEMORY_ONLY)

In [ ]:
RDD2 = RDD1.map(lambda x : remove_html_tags(x[1]))\
.map(lambda x : contractions_word(x))\
.map(lambda x: remove_emoticons(x))\
.persist(StorageLevel.MEMORY_ONLY)
# .map(lambda x : sent_tokenize(x))\


In [ ]:
RDD3 = RDD2.zipWithIndex()\
.map(lambda x: sentence_tokenize(x[0], x[1]))\
.flatMapValues(f)\
.persist(StorageLevel.MEMORY_ONLY)

In [ ]:
RDD3.map(lambda x: (x[1],)).toDF().show(100,truncate=False)

22/10/05 19:24:33 WARN BlockManager: Task 715 already completed, not releasing lock for rdd_59_0


22/10/05 19:24:37 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 27 (TID 716): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                                                                                                                                                                           

#### 1) Remove Emoji, URL, Phone Number, Currency, Punctuations, Digit, E-mail
#### 2) To Lower

In [ ]:
from cleantext import clean

def clean_text(text): 
    return str(clean(text,
            fix_unicode=True,               # fix various unicode errors
            to_ascii=False,                  # transliterate to closest ASCII representation
            lower=True,                     # lowercase text
            no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
            no_urls=True,                  # replace all URLs with a special token
            no_emails=True,                # replace all email addresses with a special token
            no_phone_numbers=True,         # replace all phone numbers with a special token
            no_numbers=True,               # replace all numbers with a special token
            no_digits=False,                # replace all digits with a special token
            no_currency_symbols=True,      # replace all currency symbols with a special token
            no_punct=True,                 # remove punctuations
            no_emoji=True,
            replace_with_punct="",          # instead of removing punctuations you may replace them
            replace_with_url="",
            replace_with_email="",
            replace_with_phone_number="",
            replace_with_number="",
            replace_with_digit="",
            replace_with_currency_symbol="",
            lang="en"                       # set to 'de' for German special handling
            ))

# RDD2 = RDD.map(lambda x: remove_emoticons(x))\
# .map(lambda x : (clean_text(x)))\
# .filter(lambda x : x != '')\
# .persist(StorageLevel.MEMORY_ONLY)

#### Remove Symbols

In [ ]:
def remove_symbols(text):
    symbols = ['+', '^', '|', '~', '>', '<', '=', '`']
    for x in symbols:
        text = text.replace(x, ' ')
    return text

In [ ]:
RDD4 = RDD3.map(lambda x: (x[1],))\
.map(lambda x : (clean_text(x)))\
.filter(lambda x : x != '')\
.map(lambda x : (remove_symbols(x)))\
.filter(lambda x : x != '')\
.persist(StorageLevel.MEMORY_ONLY)

In [ ]:
df3 = RDD4.map(lambda x: (x, )).toDF()

22/10/05 19:25:51 WARN BlockManager: Task 717 already completed, not releasing lock for rdd_67_0


In [ ]:
df3.show(truncate=False)

22/10/05 19:25:55 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 31 (TID 718): Attempting to kill Python Worker
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
from pyspark.sql.functions import col, explode, regexp_replace, length

new_df = df3.select("_1")\
    .where(col("_1").isNotNull()) \

#.where(length(col("_1")) >= 900)

new_df.show(10,truncate=False)

22/10/05 19:26:45 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 39 (TID 722): Attempting to kill Python Worker
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                               |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|hebron valley grange no                                                                                                                                          |
|also known as west hebron methodist church is a historic grange building located in the hamlet of west hebron in the town of hebron in washington county new york|
|it was originally built about as a methodi

In [ ]:
import re

new_df.rdd.map(lambda x: (re.sub(r'[^a-zA-Z0-9 ]', '', x[0]), ))\
    .toDF().show(100,truncate=False)

22/10/05 19:26:54 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 41 (TID 723): Attempting to kill Python Worker


22/10/05 19:26:55 WARN BlockManager: Task 724 already completed, not releasing lock for rdd_67_0
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
final_df = new_df.rdd.map(lambda x: (re.sub(r'[^a-zA-Z0-9 ]', '', x[0]), ))\
    .setName("TestSetName")\
    .persist(StorageLevel.MEMORY_ONLY)\
    .toDF()

22/10/05 19:27:25 WARN BlockManager: Task 725 already completed, not releasing lock for rdd_101_0


In [ ]:
final_df.show(20,truncate=False)

22/10/05 19:27:40 WARN BlockManager: Task 726 already completed, not releasing lock for rdd_101_0
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
final_df.count()

12721064

In [ ]:
final_df = final_df.coalesce(1)

In [ ]:
final_df.write.parquet("hdfs://g5.bigtop.it:8020/user/root/wikidata_en_preprocessed.parquet") 

In [ ]:
df_test_par = spark.read.parquet("hdfs://g5.bigtop.it:8020/user/root/wikidata_en_preprocessed.parquet")

In [ ]:
df_test_par.show(60,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  